In [1]:
import pandas as pd

In [9]:
df = pd.read_csv("E:\\WorkSpace\\Movie Recommended\\input\\train_v2.csv")

In [10]:
df

,ID,user,movie,rating
0,610739,3704,3784,3
1,324753,1924,802,3
2,808218,4837,1387,4
3,133808,867,1196,4
4,431858,2631,3072,5
...,...,...,...,...
750151,259179,1586,1077,5
750152,365839,2129,2700,5
750153,131933,854,3102,3
750154,671156,4033,3479,5


In [11]:
df.user.nunique()

6040

In [14]:
df.movie.nunique()

3676

In [15]:
df.shape

(750156, 4)

In [1]:
import tez
import pandas as pd
from sklearn import model_selection
import torch
import torch.nn as nn
from sklearn import metrics, preprocessing
import numpy as np

class MovieDataset:
    def __init__(self, users, movies, ratings):
        self.users = users
        self.movies = movies
        self.ratings = ratings

    def __len__(self):
        return len(self.users)

    def __getitem__(self, item):
        user = self.users[item]
        movie = self.movies[item]
        rating = self.ratings[item]

        return {
            "user": torch.tensor(user, dtype=torch.long),
            "movie": torch.tensor(movie, dtype=torch.long),
            "rating": torch.tensor(rating, dtype=torch.long)
        }

class RecSysModel(tez.Model):
    def __init__(self, num_users, num_movies):
        super().__init__()
        self.user_embed = nn.Embedding(num_users, 32)
        self.movie_embed = nn.Embedding(num_movies, 32)
        self.out = nn.Linear(64, 1)
        self.step_scheduler_after = "epoch"

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=3, gamma=0.7)
        return sch

    def monitor_metrics(self, output, rating):
        output = output.detach().cpu().numpy()
        rating = rating.detach().cpu().numpy()

        return {
            'rmse': np.sqrt(metrics.mean_squared_error(rating, output))
        }

    def forward(self, users, movies, ratings):
        user_embeds = self.user_embed(users)
        movie_embeds = self.movie_embed(movies)
        output = torch.cat([user_embeds, movie_embeds], dim=1)

        loss = nn.MSELoss()(output, ratings.view(-1, 1))
        calc_metrics = self.monitor_metrics(output, ratings.view(-1, 1))

        return output, loss, calc_metrics


c:\Users\loadi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train():
    df = pd.read_csv("../input/train_v2.csv")

    lbl_user = preprocessing.LabelEncoder()
    lbl_movie = preprocessing.LabelEncoder()

    df.user = lbl_user.fit_transform(df.user.values)
    df.movie = lbl_movie.fit_transform(df.movie.values)

    df_train, df_valid = model_selection.train_test_split(
        df, test_size=0.1, random_state=42, stratify=df.rating.values)

    train_dataset = MovieDataset(
        users=df_train.user.values, movies=df_train.movie.values, ratings=df_train.rating.values)

    valid_dataset = MovieDataset(
        users=df_valid.user.values, movies=df_valid.movie.values, ratings=df_valid.rating.values)

    model = RecSysModel(num_users=len(lbl_user.classes_),
                        num_movies=len(lbl_movie.classes_))
    model.fit(
        train_dataset, valid_dataset, train_bs=1024, valid_bs=1024, fp16=True
    )

if __name__ == "__main__":
    train()

NOTE: This is old Model class and is deprecated. It will no longer be maintained! Please use version > 0.5.1. Its much better and supports multi-gpu training too!


AssertionError: Torch not compiled with CUDA enabled